This notebook is an example of how to create in Bokeh 2 different plot functions which have their variables interactively linked to 4 widget sliders. The Jupyter notebook viewer link is here http://nbviewer.jupyter.org/github/alitarraf/JupyterWidgets/blob/master/4sliders2plotsBokeh.ipynb

In [50]:
import numpy as np
from math import sin, radians,pi
from bokeh.layouts import layout
from bokeh.models import CustomJS, Slider, ColumnDataSource, WidgetBox
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column, row

output_notebook()


def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)
    
    rho=1.225
    n=30
    Q = np.arange(0.0, 300, 10/100.0)
    #DP=0.5*rho*((pi*d2*n)**2-(Q/(pi*d2*b2*sin(radians(Beta2))))**2)
    DP=Q
    
    source = ColumnDataSource(data=dict(x=x, y=y))
    sourceDP=ColumnDataSource(data=dict(Q=Q, DP=DP))

    plot = figure(y_range=(-10, 10),plot_width=300,
                  plot_height=400,title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    plotDP = figure(y_range=(-100, 300),plot_width=300, plot_height=400,
                   title="Sliders example DP")
    plotDP.line('Q', 'DP', source=sourceDP, line_width=3, line_alpha=0.6)
    
    callback = CustomJS(args=dict(source=source,sourceDP=sourceDP), code="""
        var data = source.data;
        var dataDP =sourceDP.data;
        
        var OD = OD.value;
        var Beta2 = Beta2.value;
        var B2 = B2.value;
        var Qt = Qt.value;
        
        x = data['x']
        y = data['y']
        Q = dataDP['Q']
        DP = dataDP['DP']
        
        for (i = 0; i < x.length; i++) {
            y[i] = Qt + OD*Math.sin(Beta2*x[i]+B2);
        }
        
        for (i = 0; i < Q.length; i++) {
            DP[i] = 0.5*(Math.pow((Math.PI*B2),2)-Math.pow((Q[i]/(Math.PI*OD*B2*Math.sin(Beta2*Math.PI/180))),2));
        }
        
        source.change.emit();
        sourceDP.change.emit();
    """)

    OD_slider = Slider(start=1.2, end=10, value=1, step=.1, title="OD", callback=callback, callback_policy='mouseup')
    callback.args["OD"] = OD_slider

    Beta2_slider = Slider(start=5, end=10, value=1, step=.1, title="Beta2", callback=callback)
    callback.args["Beta2"] = Beta2_slider

    B2_slider = Slider(start=1, end=6.4, value=1, step=.1, title="B2", callback=callback)
    callback.args["B2"] = B2_slider

    Qt_slider = Slider(start=-5, end=5, value=1, step=.1, title="Qt", callback=callback)
    callback.args["Qt"] = Qt_slider

    widgets = WidgetBox(OD_slider, Beta2_slider, B2_slider, Qt_slider)
    return [widgets, plot,plotDP]


l=row(slider())
#l=layout(slider())

show(l)


Loading BokehJS ...